In [1]:
import duckdb
con = duckdb.connect('../../wd41_4d_v55.db')

In [2]:
con.query('select count(*) from results')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     64329194 │
└──────────────┘

In [3]:
con.execute("copy (select * from results limit 10000) to 'results.parquet' (format parquet)")

IOException: IO Error: Cannot open file "results.parquet": Is a directory

In [10]:
con.execute("""
install httpfs;
load httpfs;
set s3_region='us-east-1';
set s3_access_key_id='';
set s3_secret_access_key='';
""")

In [16]:
%%time
con.execute("copy (select * from results where step_id > 7 and step_id <= 10) to 'tmp.parquet' (FORMAT parquet)")

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

CPU times: user 14.5 s, sys: 630 ms, total: 15.2 s
Wall time: 15.2 s


In [27]:
%%time
df = con.query('select * from results where step_id > 7 and step_id <= 10').df()
df.shape[0]

CPU times: user 4.44 s, sys: 4.06 s, total: 8.51 s
Wall time: 7.68 s


4295117

In [23]:
%%time
arrow_tbl = con.query('select * from results where step_id > 7 and step_id <= 10').arrow()

CPU times: user 5.13 s, sys: 1.02 s, total: 6.15 s
Wall time: 3.57 s


In [28]:
%%time
df.to_csv('tmp.csv', index=False)

CPU times: user 5min 43s, sys: 8.61 s, total: 5min 51s
Wall time: 5min 53s


In [6]:
import time
import asyncio
import confirm.cloud.clickhouse as ch
import clickhouse_connect
con_details = ch.get_ch_config()
con_details['database'] = 'unnamed_7483655fa9c1418fa12f79915a9bf545'
ch_client = clickhouse_connect.get_client(**con_details, connect_timeout=60)
name = "results"
cols = ch.get_create_table_cols(df)
ch.command(
    ch_client,
    f"""
                CREATE TABLE IF NOT EXISTS {name} ({",".join(cols)})
                ENGINE = MergeTree()
                ORDER BY ()
                """,
    ch.default_insert_settings,
)


2023-03-27T20:42:53-0400 Code: 62. DB::Exception: Syntax error: failed at position 54 ('0') (line 2, col 53): 0 Nullable(Float64),1 Nullable(Float64),2 Nullable(Float64),3 Nullable(Float64),4 Nullable(Float64),5 Nullable(Float64),6 Nullable(Float64),7 Nullable(Float64),. Expected one of: columns or indices declaration list, list of elements, table property (column, index, constraint) declaration, INDEX, CONSTRAINT, PROJECTION, PRIMARY KEY, column declaration, identifier. (SYNTAX_ERROR) (version 22.13.1.24479 (official build))



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/mt/cmys2v_143q1kpcrdt5wcdyr0000gn/T/ipykernel_21669/1814994783.py:10 in <module>    │
│                                                                                                  │
│                                                                                                  │
│ /Users/tbent/confirm/confirmasaurus/confirm/cloud/clickhouse.py:330 in command                   │
│                                                                                                  │
│   329 │   start = time.time()                                                                    │
│ ❱ 330 │   out = retry_ch_action(client.command, query, *args, **kwargs)                          │
│   331 │   logger.debug(f"Command took {time.time() - start} seconds\n{query}")                   │
│                                                                                                  │
│ /Users/tbent/confirm/confirmasaurus/confirm/cloud/clickhouse.py:265 in retry_ch_action           │
│                                                                                                  │
│   264 │   │                                                                                      │
│ ❱ 265 │   │   raise e                                                                            │
│   266                                                                                            │
│                                                                                                  │
│ /Users/tbent/confirm/confirmasaurus/confirm/cloud/clickhouse.py:245 in retry_ch_action           │
│                                                                                                  │
│   244 │   try:                                                                                   │
│ ❱ 245 │   │   return method(*args, **kwargs)                                                     │
│   246 │   except clickhouse_connect.driver.exceptions.DatabaseError as e:                        │
│                                                                                                  │
│ /Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/clickhouse_connect/driver/htt │
│ pclient.py:288 in command                                                                        │
│                                                                                                  │
│   287 │   │   method = 'POST' if payload else 'GET'                                              │
│ ❱ 288 │   │   response = self._raw_request(payload, params, headers, method)                     │
│   289 │   │   result = response.data.decode()[:-1].split('\t')                                   │
│                                                                                                  │
│ /Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/clickhouse_connect/driver/htt │
│ pclient.py:363 in _raw_request                                                                   │
│                                                                                                  │
│   362 │   │   │   │   │   error_handler(response)                                                │
│ ❱ 363 │   │   │   │   self._error_handler(response)                                              │
│   364                                                                                            │
│                                                                                                  │
│ /Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/clickhouse_connect/driver/htt │
│ pclient.py:304 in _error_handler                                                                 │
│                                                                                                  │
│   303 │   │   │   err_str = f':{err_str}\n {err_msg[0:240]}

In [58]:
start = time.time()
n = 10000
tasks = [
    asyncio.to_thread(
        ch_client.insert_arrow,
        "results",
        arrow_tbl[i * n : (i + 1) * n],
        settings=ch.async_insert_settings,
    )
    for i in range(64)
]
await asyncio.gather(*tasks)
print(time.time() - start)


CancelledError

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/mt/cmys2v_143q1kpcrdt5wcdyr0000gn/T/ipykernel_17085/3310388982.py:12 in <module>    │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
CancelledError

In [15]:
%%time
con.execute("copy (select * from results where step_id > 7 and step_id <= 10) to 's3://confirmasaurus-db-backup/results' (FORMAT parquet, PARTITION_BY ('step_id'), ALLOW_OVERWRITE TRUE)")

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

CPU times: user 26.8 s, sys: 3.03 s, total: 29.9 s
Wall time: 48.6 s


In [53]:
df.shape

(4295117, 130)

In [60]:
ch_client.database

'unnamed_7483655fa9c1418fa12f79915a9bf545'

In [3]:
import numpy as np
import pandas as pd
import pyarrow as pa
arr = np.random.uniform(size=(400000, 130))
df = pd.DataFrame(arr)
arrow_tbl = pa.Table.from_pandas(df)

import duckdb
import time
con = duckdb.connect()
con.execute("""
install httpfs;
load httpfs;
set s3_region='us-east-1';
set s3_access_key_id='';
set s3_secret_access_key='';
""")
start = time.time()
con.execute("copy (select * from arrow_tbl) to 's3://confirmasaurus-db-backup/results' (FORMAT parquet)")
print('took ', time.time() - start)
print('took ', time.time() - start)
print('took ', time.time() - start)
print('took ', time.time() - start)
print('took ', time.time() - start)

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

took  43.81673884391785
took  43.81678104400635
took  43.81679892539978
took  43.8168158531189
took  43.81683397293091


In [4]:
import confirm.cloud.modal_util as modal_util
import modal
import numpy as np
import pandas as pd
import pyarrow as pa
modal_cfg = modal_util.get_defaults()

stub = modal.Stub()

@stub.function(**modal_cfg)
def test_fnc():
    arr = np.random.uniform(size=(400000, 130))
    df = pd.DataFrame(arr)
    arrow_tbl = pa.Table.from_pandas(df)

    import duckdb
    import time
    con = duckdb.connect()
    con.execute("""
    install httpfs;
    load httpfs;
    set s3_region='us-east-1';
    set s3_access_key_id='';
    set s3_secret_access_key='';
    """)
    start = time.time()
    con.execute("copy (select * from arrow_tbl) to 's3://confirmasaurus-db-backup/results' (FORMAT parquet)")
    print('took ', time.time() - start)
    print('took ', time.time() - start)
    print('took ', time.time() - start)
    print('took ', time.time() - start)
    print('took ', time.time() - start)
    
    # ch_client = ch.connect('unnamed_7483655fa9c1418fa12f79915a9bf545')
    # async def insert():
    #     n = 10000
    #     tasks = [
    #         asyncio.to_thread(
    #             ch_client.insert_arrow,
    #             "results",
    #             arrow_tbl[i * n : (i + 1) * n],
    #             settings=ch.async_insert_settings,
    #         )
    #         for i in range(64)
    #     ]
    #     await asyncio.gather(*tasks)
    # asyncio.run(insert())

with stub.run():
    test_fnc.call()

Output()

✓ Initialized. View app at https://modal.com/apps/ap-Md2LcMuNqrnpoiccl08Tg1

Output()

2023-03-27T17:20:43-0400 Mount of '.' is empty.


✓ Created objects.
├── 🔨 Created test_fnc.
├── 🔨 Created mount /Users/tbent/confirm/confirmasaurus/confirm
├── 🔨 Created mount /Users/tbent/confirm/confirmasaurus/imprint/imprint
└── 🔨 Created mount .

Output()

took  3.69688081741333
took  3.696958065032959
took  3.6969656944274902
took  3.696972608566284


took  3.6969923973083496


✓ App completed.